In [1]:
import tensorflow as tf
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
tf.logging.set_verbosity(tf.logging.ERROR)

steps   = 28
indims  = 28
neurons = 150
outdims = 10

learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, steps, indims])
y = tf.placeholder(tf.int32, [None])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units = neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype = tf.float32)

logits = tf.layers.dense(states, outdims)
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits)

loss        = tf.reduce_mean(cross_entropy)
optimizer   = tf.train.AdamOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(loss)
correct     = tf.nn.in_top_k(logits, y, 1)
accuracy    = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/")
X_test = mnist.test.images.reshape((-1, steps, indims))
y_test = mnist.test.labels

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
epochs = 10
batch_size = 150

with tf.Session() as sess:
    init.run()
    
    for epoch in range(epochs):
        num_batches = mnist.train.num_examples // batch_size
        for iteration in range(num_batches):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape((-1, steps, indims))
            sess.run(training_op, feed_dict = {X: X_batch, y: y_batch})
        
        train_accuracy = accuracy.eval(feed_dict = {X: X_batch, y: y_batch})
        test_accuracy  = accuracy.eval(feed_dict = {X: X_test, y: y_test})
        print(epoch, "Train Accuracy: %0.4f, Test Accuracy: %0.4f" % (train_accuracy, test_accuracy))

0 Train Accuracy: 0.9200, Test Accuracy: 0.9069
1 Train Accuracy: 0.9467, Test Accuracy: 0.9434
2 Train Accuracy: 0.9800, Test Accuracy: 0.9433
3 Train Accuracy: 0.9400, Test Accuracy: 0.9598
4 Train Accuracy: 0.9667, Test Accuracy: 0.9638
5 Train Accuracy: 0.9800, Test Accuracy: 0.9614
6 Train Accuracy: 0.9867, Test Accuracy: 0.9652
7 Train Accuracy: 0.9800, Test Accuracy: 0.9728
8 Train Accuracy: 0.9800, Test Accuracy: 0.9648
9 Train Accuracy: 0.9867, Test Accuracy: 0.9699
